# Tools for Analyzing Time Series of Satellite Imagery (TATSSI)

## Download data from the GEE
TATSSI allows to dowload data products from Google Earth Engine [GEE](https://developers.google.com/earth-engine/datasets/) and create and process time series from different sensors such as:

* Moderate Resolution Imaging Spectroradiometer [MODIS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#MODIS_anchor)
* Visible Infrared Imaging Radiometer Suite [VIIRS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/#S-NPPVIIRS_anchor)

Downloading products from the GEE requires a Google Account, please, first regsiter as a user to get the login credentials and follow the instructions to authenticate to the Earth Engine using the [Python API](https://developers.google.com/earth-engine/python_install).

1. Select your are of study, you can save it as a GeoJSON file, e.g. from the [geojson.io](http://geojson.io/) site.
2. For the time being you need to match the official MODIS product bands and the GEE bands since the band names are not consistent.
3. Generate the time series, you can then use the rest of the (awesome) TATSSI tools!

In [2]:
# Import libraries
import os
import json

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from helpers import utils
from TATSSI.download.gee_downloader import export_collection
from TATSSI.time_series.generator import Generator

In [3]:
# Area of study
with open('coahuila.geojson') as json_file:
    region = json.load(json_file)
    
region = region['features'][0]['geometry']['coordinates']
geometry = ee.Geometry.Polygon(region)
print(geometry)

ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        -103.32092285156249,
        27.768620944918666
      ],
      [
        -101.0577392578125,
        27.768620944918666
      ],
      [
        -101.0577392578125,
        29.46829664171322
      ],
      [
        -103.32092285156249,
        29.46829664171322
      ],
      [
        -103.32092285156249,
        27.768620944918666
      ]
    ]
  ],
  "evenOdd": true
})


For this example we will use the MODIS MOD13A2 product. Match the band names from the [LPDAAC](https://lpdaac.usgs.gov/products/mod13a2v006/) and the [GEE](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2).

In [7]:
# Dictionary with "GEE" : "LPDAAC"
bands = {"DayOfYear"       : "1_km_16_days_composite_day_of_the_year",
         "EVI"             : "1_km_16_days_EVI",
         "RelativeAzimuth" : "1_km_16_days_relative_azimuth_angle",
         "SummaryQA"       : "1_km_16_days_pixel_reliability",
         "sur_refl_b02"    : "1_km_16_days_NIR_reflectance",
         "sur_refl_b07"    : "1_km_16_days_MIR_reflectance",
         "DetailedQA"      : "1_km_16_days_VI_Quality",
         "NDVI"            : "1_km_16_days_NDVI",
         "SolarZenith"     : "1_km_16_days_sun_zenith_angle",
         "sur_refl_b01"    : "1_km_16_days_red_reflectance",
         "sur_refl_b03"    : "1_km_16_days_blue_reflectance",
         "ViewZenith"      : "1_km_16_days_view_zenith_angle"}

Set the download parameters

In [17]:
# Dates as YYYY-MM-DD
start_date, end_date = '2008-01-01', '2020-12-01'

# Get MODIS data
product_name = 'MOD13A2'
version = '006'

# Product using GEE nomenclature
product = f'MODIS/{version}/{product_name}'

# Get GEE collection
ee_collection = (ee.ImageCollection(product).
                 filterBounds(geometry).
                 filterDate(start_date, end_date))

# Output directory
output_dir = '/home/glopez/Projects/TATSSI/data/test_gee'

Get the data!

In [18]:
# Export image collection
datasets = export_collection(ee_collection, output_dir, bands=bands,
                             product=product, version=version,
                             scale=None, crs='EPSG:4326', region=region,
                             file_per_band=True)

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Total number of images: 297

INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 1/297: 2008_01_01.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/92af76f2bfcefdf0f3473898f90f3aa6-f7944f190bd67b2c5ef18fe5d06da15c:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/test_gee
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 2/297: 2008_01_17.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0ae89cb626f3c0d3b8e898c580331816-e15d843dfdd81c3334fcb736f991fc21:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/test_gee
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 3/297: 2008_02_02.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


...

INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/dfac1f9a6f33f4e8b8caa5ec0ae103b1-f554ca9e20fd3e2f2c4b780fdb785f84:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/test_gee
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 296/297: 2020_10_31.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/fe1e6675e1da31dd8e8929cd5714675f-97bbc2591e4f60c86249667ed0edccb6:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/test_gee
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
INFO:TATSSI.download.gee_downloader:Exporting 297/297: 2020_11_16.tif
INFO:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails?fields=name&alt=json


Generating URL ...


INFO:TATSSI.download.gee_downloader:Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/16ee0d7306bd9aad0a556395330a761e-670956aad915dee2be22159afe5ca6e0:getPixels
Please wait ...
INFO:TATSSI.download.gee_downloader:Data downloaded to /home/glopez/Projects/TATSSI/data/test_gee


Create TATSSI time series.

In [19]:
# TATSSI Time Series Generator object
tsg = Generator(source_dir=output_dir, product=product_name,
                version=version, data_format='tif',
                progressBar=None, preprocessed=True)

# Create layerstacks
tsg._Generator__datasets = datasets
for dataset in tsg._Generator__datasets:
    tsg._Generator__generate_layerstack(dataset, 'tif')

# For the associated product layers, decode the corresponding QA bands
tsg._Generator__decode_qa('tif')

INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_NDVI created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_EVI created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_VI_Quality created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_red_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_NIR_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_blue_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_MIR_reflectance created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_view_zenith_angle created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_sun_zenith_angle created successfully.
INFO:TATSSI.time_series.generator:Layer stack for 1_km_16_days_relative_azimuth_angle created successfully.
INFO:TA

INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_VI_Quality/MOD13A2.2008_05_24.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.qu

INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_VI_Quality/MOD13A2.2008_10_31.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA la

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_VI_Quality/MOD13A2.2009_04_07.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aeros

INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aerosol Quantity...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Adjacent cloud detected...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Atmosphere BRDF Correction...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Mixed Clouds...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Land/Water Mask...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible snow/ice...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Possible shadow...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_VI_Quality/MOD13A2.2009_09_14.006.1_km_16_days_VI_Quality.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer VI Usefulness...
INFO:TATSSI.qa.EOS.quality:Decoding QA layer Aeros

...

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2016_01_17.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2016_02_02.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2016_02_18.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INF

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2017_03_06.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2017_03_22.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2017_04_07.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INF

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2018_04_23.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2018_05_09.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2018_05_25.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INF

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2019_06_10.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2019_06_26.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2019_07_12.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INF

INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2020_07_27.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2020_08_12.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INFO:TATSSI.qa.EOS.quality:Decoding MOD13A2.006...
INFO:TATSSI.qa.EOS.quality:File /home/glopez/Projects/TATSSI/data/test_gee/1_km_16_days_pixel_reliability/MOD13A2.2020_08_28.006.1_km_16_days_pixel_reliability.tif
INFO:TATSSI.qa.EOS.quality:Decoding QA layer MODLAND...
INFO:TATSSI.qa.EOS.quality:Decoding finished.
INF